# Usage 
This notebook visualizes how CXIL can be utilized. 

In [ ]:
from captum.attr import InputXGradient
import warnings
from CXIL.caipi.Transformer import MeanAndStdTransformer, randomTransformer
from CXIL.Benchmarking.metrics.Helper import get_mask
from CXIL.Data.DataLoader import load_data_and_sim,timeseries
from CXIL.Learning import LearnerStep, Replay
from CXIL.Models.SimpleCNN import SimpleCNN,CNN
from CXIL.Models.SimpleModel import SimpleMLP
from CXIL.Models.Simple1DResNet import ResNetBaseline
from CXIL.caipi.CAipi_without_TKadd import caipi
from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset
import numpy as np

## Load Data

The dataset should be a numpy array and the labels ordinal encoded. 

In [ ]:
# Load the Data
dataset ='tabular'
X,y,X_train,y_train,X_test,y_test, simulation= load_data_and_sim(dataset)

## Initialize Model 
Initalize the model you want to use for the task, the models weigjhts can be untrained or trained.

In [ ]:
# Initialize a Model 
model = SimpleMLP(10, 15)

# Build the CXIL Learner 

The Basic learner can be any of the continous algorithms contained in this Package. For the exact parameter option, please refer to the code documentation.


Learner Options: 
- MAS.MAS
- Synaptic_Intelligence.Synaptic_Intelligence
- EWC.Elastic_Weight_Regularizer

In [ ]:
# Choose a Learner
learn = LearnerStep.Basic(model)

In [ ]:
# Choose a Replay Strategy -- Not necessary, if you do not want to use replay.
learn_replay=Replay.ReplayStrategyLearner(learn, num_classes=10,mem_size = 100,batch_size= 32)

In [ ]:
# Choose the Interaction Type
# For RRR
#loss= LearnerStep.RRR_Learner(InputXGradient,simulation,model)
#int_train= RightRightReasons.RRR(model,learn,test_data=(X_test,y_test),simulation_logic=simulation,silent=0)
# For CAIPI
cai=caipi(model,  learn_replay,transformer=MeanAndStdTransformer(model,num_samples=1), evaluate_data= (X_test,y_test),silent=1,simulation_logic=simulation)

## Learn a Sample

In [ ]:
# Run
acc,f1, precision, recall, loss_calc_train, loss_calc_test=cai.iterate(X[0][np.newaxis,:],np.array([y[0]]), taskid=None) 